In [ ]:
# Obtain Canvas access token from .env file

import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
token = os.environ.get("TOKEN")

In [ ]:
import requests
import json
import pandas as pd

In [ ]:
PROVIDER = 'swinburne'

# Overhead

In [ ]:
from requests.exceptions import HTTPError
from datetime import datetime
from tzlocal import get_localzone

def parse_iso_timestamps(data : pd.DataFrame) -> pd.DataFrame:
    """
    Replace all string timestamps in a DataFrame with datetime objects.
    All timestamps must comply with ISO-8601.

    Args:
        data (DataFrame): DataFrame containing ISO-8601 string timestamps.

    Returns:
        data (DataFrame): DataFrame containing datetime timestamps.
    """

    # Obtain all columns which can be converted
    # into a timestamp through ugly brute-forcing
    timestamp_columns = []
    for key, value in data.iloc[0].to_dict().items():
        try:
            datetime.fromisoformat(value)
            timestamp_columns.append(key)
        except:
            pass

    def isotime_to_timestamp(value : str | None, use_local_timezone : bool = True, as_string : bool = False):
        if not value: return None
        
        time = datetime.fromisoformat(value)
        if use_local_timezone:
            time = time.astimezone(get_localzone())

        if as_string:
            time = time.strftime("%A, %d %B %Y, %I:%M %p")
            
        return time
    
    for column in timestamp_columns:
        data[column] = data[column].map(isotime_to_timestamp)

    return data
    
def decode_canvas_response(response : requests.Response) -> pd.DataFrame:
    """
    Converts raw Canvas API response into a DataFrame for downstream use,
    or raises Exception if the response is invalid.

    Args:
        response (Response): The raw API response obtained from requests.

    Returns:
        response_data (DataFrame): The response data.
    """
    
    if response.ok:
    
        # Canvas API will always respond in JSON format,
        # but we obtain the response in bytes, so it
        # must be decoded into a raw string and then
        # encoded into a JSON object.
        
        response_data = response.content.decode('utf-8')
        
        response_data = json.loads(response_data)

        if not response_data:
            # Empty response
            return pd.DataFrame([])
        
        if type(response_data) is not list: response_data = [response_data]
        
        response_data = pd.DataFrame(response_data)

        response_data = parse_iso_timestamps(response_data)
    
    else:
        response.raise_for_status()

    return response_data

# Get current units

In [ ]:
# Two enrolment states:
# active   - Visible on home menu
# complete - Invisible

units = requests.get(f"https://swinburne.instructure.com/api/v1/courses/", params={
    "access_token":token,
    "enrollment_state":"active"
})
units = decode_canvas_response(units)

In [ ]:
# Internally, Swinburne Organisation (ORG) units are assigned to enrollment term ID 1
# Since we're only concerned with academic units, let's filter out organisation units
units = units.loc[units.enrollment_term_id != 1]

# Get assignments for current units

In [ ]:
# Bucket (Status)
# UPCOMING:  unsubmitted
# OVERDUE:   overdue
# COMPLETE:  past

assignments = []

for unit_id in units["id"].to_list():

    response = requests.get(f"https://swinburne.instructure.com/api/v1/courses/{unit_id}/assignments", params={
        "access_token":token,
        "order_by":"due_at",
        "bucket":"unsubmitted",
        "include":"submission"
    })
    
    response = decode_canvas_response(response)
    response = response.to_dict(orient="records")
    assignments.extend(response)

assignments = pd.DataFrame(assignments)

In [ ]:
assignments["course_name"] = assignments["course_id"].map( lambda x: units.loc[units.id == x].name[0])

In [ ]:
assignments[["id", "course_name", "name", "due_at"]]